## Hands on session 2.4 - highly composite derivations

This session returns to the calculation tree derivations first shown in session 1.3, building on these tools and demonstrating a real use case for them.

The file `anisotropy.py` is supplied which contains a collection of functions used in more complex derivations.

Demonstrated concepts:

- Building functions out of node calcu   lations
- Appyling range filter cutoffs in derivations
- Additive derivations
- Implicit temperature derivations


In [18]:
import numpy as np
import xarray as xr
import sys

import holoviews as hv
import matplotlib.pyplot as plt
import matplotlib as mpl
from holoviews import opts
import panel as pn

import anisotropy as aniso
import RMK_support.simple_containers as sc
import RMK_support.common_models as cm 
import RMK_support.IO_support as io
from RMK_support import RKWrapper, Grid, treeDerivation, Node
import RMK_support.dashboard_support as ds

### The system
A true model of an anisotropic plasma would require a full set of fluid equations. For the sake of brevity this is not done here. Nonetheless, one integral part of an anisotropic plasma model is the isotropisation of temperatures. It is this process that is demonstrated here in this notebook.

The equations shown here can be found in [Chodura et al.](link)

We will look at the simplified one species case (for electrons) first. In this case, the anisotropy is quantified by the variable 

$$
    X = \frac{T_\perp}{T_\parallel} - 1
$$

With these variables defined, the equations that govern the system are:
$$
    \frac{\partial W_\perp}{\partial t} = 2 n \nu \left[T_\perp (K_{002} - K_{200}) \right]
$$
$$
    \frac{\partial W_\parallel}{\partial t} = 4 n \nu \left[ T_\parallel (K_{200} - K_{002}) \right]
$$
where $\nu$ is in our case an arbitrary collisionality.
$K_{LMN}$ are results of taking moments of a bi-Maxwellian:
$$
    K_{200} = \frac{1}{X} \left[ -1 + (1 + X)\varphi (X) \right]
$$
$$
    K_{002} = \frac{2}{X} \left[ 1 - \varphi (X) \right]
$$
where $\varphi$ for $X>0$:
$$
    \varphi = \frac{\arctan{(\sqrt{X})}}{\sqrt{X}}
$$
and for $X<0$:
$$
    \varphi = \frac{\log{\frac{1+\sqrt{-X}}{1-\sqrt{-X}}}}{2\sqrt{-X}}
$$

### Basic setup

In a full model evolving all fluid variables we would include many spatial derivative terms. However, since we're only interested in the 0D effect of temperature isotropisation we can reduce the number of cells simulated as much as possible. Because we want to write the setup in a general way, using staggered grids, we need at least 2 cells.

In [19]:
#Wrapper initialization
rk = RKWrapper()

#I/O setuo
rk.jsonFilepath = "./config.json" 
hdf5Filepath = "./RMKOutput/day_2_4/"
rk.setHDF5Path(hdf5Filepath) 

#Grid initialization
xGridWidths = np.ones(2)
rk.grid = Grid(xGridWidths, interpretXGridAsWidths=True)

### Normalization

In [20]:
elMass = 9.10938e-31 # electron mass
amu = 1.6605390666e-27  # atomic mass unit

rk.setNormDensity(1.0e19)
rk.setNormTemperature(10.0)
rk.setNormRefZ(1.0)

### Handling particle species data

In [21]:
rk.addSpecies("e", 0, atomicA=elMass/amu, charge=-1.0, associatedVars=["ne", "Ge", "WePar", "WePerp"])

electronSpecies = rk.getSpecies("e")

### Base variables

In [22]:
# Set conserved variables in container
TePar = 2*np.ones(2)
TePerp = 5*np.ones(2)

ne = np.ones(2)

WePar = ne*TePar/2
WePerp = ne*TePerp

# Units are not used by ReMKiT1D, but are useful to specify for later plotting
rk.addVarAndDual("ne", ne, units='$10^{19} m^{-3}$', isCommunicated=True)
rk.addVarAndDual("Ge", primaryOnDualGrid=True, isCommunicated=True)  # Ge_dual is evolved, and Ge is derived
rk.addVarAndDual("WePar", WePar, units='$10^{20} eV m^{-3}$', isCommunicated=True)
rk.addVarAndDual("WePerp", WePerp, units='$10^{20} eV m^{-3}$', isCommunicated=True)

# Temperatures
rk.addVarAndDual("TePar", TePar, isStationary=True, units='$10eV$', isCommunicated=True)
rk.addVarAndDual("TePerp", TePerp, isStationary=True, units='$10eV$', isCommunicated=True)

### Anisotropic variables
Let us begin by declaring the necessary derived variables used in this workbook: $\nu$, $K_{200}$ & $K_{002}$.

In [23]:
rk.addVar("collFreq", np.ones(2),isDerived=True)

rk.addCustomDerivation("K200", treeDerivation(aniso.K_LMN(Node("TePerp")/Node("TePar"),"200")))
rk.addCustomDerivation("K002", treeDerivation(aniso.K_LMN(Node("TePerp")/Node("TePar"),"002")))

There is one issue with these terms, namely as the system approaches isotropy, the value of $X$ will aproach zero. This is problematic as the numerical solver will break due to a divide by zero error. Hence, the terms $K_{200}$ and $K_{002}$ need to have a form that is used only when X approaches zero. This situation is a prime candidate to make use of ReMKiT1D's `rangeFilterDerivation()` function. 

To begin, let's create the variable $X$ as well as derivations for $K_{200}$ $K_{002}$ when $X$ is small:

In [24]:
rk.addVarAndDual("X",
                 isDerived=True,
                 derivationRule=sc.derivationRule("XDeriv", ("TePar","TePerp")),
                 derivOptions=treeDerivation(Node("TePerp")/Node("TePar") - 1))

rk.addCustomDerivation("K200Small", treeDerivation(aniso.K_LMN(Node("TePerp")/Node("TePar"),"200",smallX=True)))

rk.addCustomDerivation("K002Small", treeDerivation(aniso.K_LMN(Node("TePerp")/Node("TePar"),"002",smallX=True)))

Let us take a look at how the `rangeFilterDerivation()` function works:

In [25]:
help(sc.rangeFilterDerivation)

Help on function rangeFilterDerivation in module RMK_support.simple_containers:

rangeFilterDerivation(derivName: str, controlIndices: List[int], controlRanges: List[List[float]], derivIndices: Optional[List[int]] = None) -> dict
    Return composite derivation object which wraps another derivation with range-based filtering, zeroing out all values where
    passed variables corresponding to controlIndices are outside ranges specified by controlRanges. If derivIndices aren't present all
    passed variables are passed to the derivation in that order.
    
    Args:
        derivName (str): Name of the wrapped derivation
        controlIndices (List[int]): Indices of passed variables corresponding to control variables
        controlRanges (list[List[float]]): Ranges (all length 2) corresponding to each control variable
        derivIndices (Union[None,List[int]], optional): Optional subset of passed variables passed to the wrapped derivation. Defaults to None, passing all variables.
  

We can use the `rangeFilterDerivation()` function found in `simple_containers` to write what the value of $K_{200}$ and $K_{002}$ in the different regimes. Since $-\infty < X < \infty$ we must assign three different ranges in which $K_{200}$ and $K_{002}$ must be calculated. These being:
$$
    \begin{cases}
        -1e16 < X < -0.01 \\
        |X| << 0.01 \\
        0.01 > X < 1e16 \\
    \end{cases}
$$
Note that the range is not actually $-\infty < X < \infty$, since the solver deals with floating point numbers we must give a large float as the range.

In [26]:
rk.addCustomDerivation("filterSmallValsK200", sc.rangeFilterDerivation("K200Small",controlIndices=[1],controlRanges=[[-0.01,0.01]],derivIndices=[2,3]))
rk.addCustomDerivation("filterPlusLargeK200", sc.rangeFilterDerivation("K200",controlIndices=[1],controlRanges=[[0.01,1e16]],derivIndices=[2,3]))
rk.addCustomDerivation("filterMinusLargeK200", sc.rangeFilterDerivation("K200",controlIndices=[1],controlRanges=[[-1e16,-0.01]],derivIndices=[2,3]))

rk.addCustomDerivation("filterSmallValsK002", sc.rangeFilterDerivation("K002Small",controlIndices=[1],controlRanges=[[-0.01,0.01]],derivIndices=[2,3]))
rk.addCustomDerivation("filterPlusLargeK002", sc.rangeFilterDerivation("K002",controlIndices=[1],controlRanges=[[0.01,1e16]],derivIndices=[2,3]))
rk.addCustomDerivation("filterMinusLargeK002", sc.rangeFilterDerivation("K002",controlIndices=[1],controlRanges=[[-1e16,-0.01]],derivIndices=[2,3]))

Since the ranges of the filtered functions do not overlap, we can simply sum them up and store them as a single variable so that $K_{200}$ and $K_{002}$ exist over the range $-1e16 < X < 1e16$. This can be done using the `additiveDerivation()` function found in `simple_containers`.

Let us take a look at how the function `additiveDerivation()` function works:

In [27]:
help(sc.additiveDerivation)

Help on function additiveDerivation in module RMK_support.simple_containers:

additiveDerivation(derivTags: List[str], resultPower: float, derivIndices: List[List[int]], linCoeffs: List[float] = []) -> dict
    Returns property dictionary for additive composite derivation which sums up the results of each derivation in derivTags and
    raises the corresponding result to resultPower.
    
    Args:
        derivTags (List[str]): List of derivations whose output should be added
        resultPower (float): Power to raise the result of the addition
        derivIndices (List[List[int]]]): List of index lists corresponding to each derivation in derivTags.
        linCoeffs (List[List[int]]]): List linear coefficients corresponding to each derivation in derivTags. Defaults to [] resulting in a list of ones.
    
    Returns:
        dict: Dictionary representing derivation properties
    
    Usage:
        Given a passed set of variables to the additive derivation object, each individual 

In [28]:
rk.addVarAndDual("K200",
                 isDerived=True,
                 derivationRule=sc.derivationRule("filteredK200",["X","TePar","TePerp"]),
                 derivOptions=sc.additiveDerivation(["filterSmallValsK200","filterPlusLargeK200","filterMinusLargeK200"],resultPower=1., derivIndices=[[1,2,3]]*3))

rk.addVarAndDual("K002",
                 isDerived=True,
                 derivationRule=sc.derivationRule("filteredK002",["X","TePar","TePerp"]),
                 derivOptions=sc.additiveDerivation(["filterSmallValsK002","filterPlusLargeK002","filterMinusLargeK002"],resultPower=1., derivIndices=[[1,2,3]]*3))

### Implicit temperature derivation
Fluid models in ReMKiT1D are written in conservative form. As such, the temperature is considered a derived variable. However, we can make use of the `implicitTemperatures()` function found in `common_models` to derive the temperature from the implicit fluid variables such that temperature is also an implicit variable.

In [29]:
help(cm.implicitTemperatures)

Help on function implicitTemperatures in module RMK_support.common_models:

implicitTemperatures(modelTag: str, speciesFluxes: List[str], speciesEnergies: List[str], speciesDensities: List[str], speciesTemperatures: List[str], species: List[RMK_support.simple_containers.Species], speciesDensitiesDual: Optional[List[str]] = None, evolvedXU2Cells: Optional[List[int]] = None, ignoreKineticContribution=False, degreesOfFreedom: int = 3) -> RMK_support.simple_containers.CustomModel
    Generate implicit temperature derivation terms for each species
    
    Args:
        speciesFluxes (List[str]): Names of evolved species fluxes
        speciesEnergies (List[str]): Names of species energies
        speciesDensities (List[str]): Names of species densities
        speciesTemperatures (List[str]): Names of species temperature
        species (list[sc.Species]): Species objects for each species
        speciesDensitiesDual (Union[List[str],None], optional): Names of species densities on dual gri

In [30]:
# Initializing model
implicitTempModelPar = cm.implicitTemperatures(modelTag="implicitTempElPar",
                                            speciesFluxes=["Ge_dual"],
                                            speciesDensities=["ne"],
                                            speciesEnergies=["WePar"],
                                            speciesTemperatures=["TePar"],
                                            species=[electronSpecies],
                                            speciesDensitiesDual=["ne_dual"],
                                            degreesOfFreedom=1)

rk.addModel(implicitTempModelPar)

# Initializing model
implicitTempModelPerp = cm.implicitTemperatures(modelTag="implicitTempElPerp",
                                            speciesFluxes=["Ge_dual"],
                                            speciesDensities=["ne"],
                                            speciesEnergies=["WePerp"],
                                            speciesTemperatures=["TePerp"],
                                            species=[electronSpecies],
                                            speciesDensitiesDual=["ne_dual"],
                                            degreesOfFreedom=2,
                                            ignoreKineticContribution=True)

rk.addModel(implicitTempModelPerp)

Checking terms in model implicitTempElPar:
   Checking term identityTermTePar
   Checking term wTermTePar
   Checking term u2TermTePar
Checking terms in model implicitTempElPerp:
   Checking term identityTermTePerp
   Checking term wTermTePerp


### Temperature Isotropisation
We have a small number of evolved variables, with many custom terms, all with diagonal stencils. This is a prime candidate for the use of the `addNodeMatrixTermModel()` function to nearly create the models in single lines.

In [31]:
perpIsoEl = cm.addNodeMatrixTermModel(rk,
                                      modelTag="perpIsoEl", 
                                      evolvedVar="WePerp",
                                      termDefs=[(2*Node("collFreq")*Node("K002")/Node("TePerp"),"ne"),
                                                (-2*Node("collFreq")*Node("K200")/Node("TePerp"),"ne")])

parIsoEl = cm.addNodeMatrixTermModel(rk,
                                      modelTag="parIsoEl", 
                                      evolvedVar="WePar",
                                      termDefs=[(4*Node("collFreq")*Node("TePerp")*Node("K200")/(Node("TePar")**2),"ne"),
                                                (-4*Node("collFreq")*Node("TePerp")*Node("K002")/(Node("TePar")**2),"ne")])

Checking terms in model perpIsoEl:
   Checking term nodeTerm_0
   Checking term nodeTerm_1
Checking terms in model parIsoEl:
   Checking term nodeTerm_0
   Checking term nodeTerm_1


### Time loop options

In [32]:
rk.addIntegrator("BE",sc.picardBDEIntegrator(nonlinTol=1e-12,absTol=10.0,convergenceVars=['TePerp','TePar']) ) # we want all evolved variables to converge

rk.setIntegratorGlobalData(initialTimestep=0.1)

rk.addTermDiagnosisForVars(["K200","K002"])

bdeStep = sc.IntegrationStep("BE")

for tag in rk.modelTags():
    bdeStep.addModel(tag)

rk.addIntegrationStep("StepBDE",bdeStep.dict())

rk.setFixedNumTimesteps(2500)
rk.setFixedStepOutput(50)

### Write config file

In [33]:
rk.writeConfigFile()

### Data analysis

In [ ]:
numFiles = 50
loadFilenames = [hdf5Filepath+f'ReMKiT1DVarOutput_{i}.h5' for i in range(numFiles+1)]
loadedData = io.loadFromHDF5(rk.varCont, filepaths=loadFilenames,
                             isXinMeters=gridObj.isLengthInMeters)
loadedData

In [ ]:
hv.extension('matplotlib')
%matplotlib inline
plt.rcParams['figure.dpi'] = 150
hv.output(size=100, dpi=150)

In [ ]:
pn.extension(comms="vscode")  # change comms if not using VSCode
dashboard = ds.ReMKiT1DDashboard(loadedData, gridObj)

dashboard.fluid2Comparison().show()
